In [1]:
from langchain_community.llms import Ollama

import tempfile
import whisper
import langid
import shutil
from moviepy.editor import VideoFileClip

#from langchain_openai.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.document_loaders import TextLoader

from langchain.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.runnables import RunnablePassthrough

## Load the model


In [2]:
model = Ollama(model='llama3.1')

## Define the output parser

In [3]:
parser = StrOutputParser() # Output parser for the model

## Define the template for the translation/summarization task

In [4]:
prompt_translation = ChatPromptTemplate.from_template("""
    Context: {context}
    Translate the following {text} from {source_language} to {target_language}.
""")

prompt_summary = ChatPromptTemplate.from_template("""
    Summarize{answer}.
""")

## Initiating whisper

In [5]:

def extract_audio_from_video(video_path, output_audio_path):
    # Extracts the audio from a video file and saves it as an audio file.
    video_clip = VideoFileClip(video_path)
    video_clip.audio.write_audiofile(output_audio_path, codec='pcm_s16le')
    video_clip.close()

def transcribe_audio(audio_path):
    # Transcribes the audio file using Whisper.
    whisper_model = whisper.load_model("small")
    return whisper_model.transcribe(audio_path, fp16=False)["text"].strip()

file_path = "french.mp3"  

if file_path.lower().endswith(('.mp4', '.mkv', '.avi')):
    audio_file = 'temp_audio.wav' 
    extract_audio_from_video(file_path, audio_file)
    transcription = transcribe_audio(audio_file)
    os.remove(audio_file)  
#elif file_path.lower().endswith(('.mp3', '.wav', '.m4a')):
#    transcription = transcribe_audio(file_path)
else:
    raise ValueError(f"The file format of {file_path} is not supported.")

transcription = transcribe_audio(file_path)

transcription_file = "transcription.txt"
with open(transcription_file, "w") as file:
    file.write(transcription)


with open(transcription_file, "r") as file:
    transcript = file.read()
print(transcript)


language = langid.classify(transcript)[0]
print("Detected Language:", language)


'\ndef extract_audio_from_video(video_path, output_audio_path):\n    # Extracts the audio from a video file and saves it as an audio file.\n    video_clip = VideoFileClip(video_path)\n    video_clip.audio.write_audiofile(output_audio_path, codec=\'pcm_s16le\')\n    video_clip.close()\n\ndef transcribe_audio(audio_path):\n    # Transcribes the audio file using Whisper.\n    whisper_model = whisper.load_model("small")\n    return whisper_model.transcribe(audio_path, fp16=False)["text"].strip()\n\nfile_path = "french.mp3"  \n\nif file_path.lower().endswith((\'.mp4\', \'.mkv\', \'.avi\')):\n    audio_file = \'temp_audio.wav\' \n    extract_audio_from_video(file_path, audio_file)\n    transcription = transcribe_audio(audio_file)\n    os.remove(audio_file)  \n#elif file_path.lower().endswith((\'.mp3\', \'.wav\', \'.m4a\')):\n#    transcription = transcribe_audio(file_path)\nelse:\n    raise ValueError(f"The file format of {file_path} is not supported.")\n\ntranscription = transcribe_audio(fi

## Breaking the transcript into chunks

In [5]:
loader = TextLoader("transcription.txt")
text_documents = loader.load()
text_documents

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap=0)
texts = text_splitter.split_documents(text_documents)

print(texts)

[Document(metadata={'source': 'transcription.txt'}, page_content='Numéro 1. Toucher un crayon. Numéro 2. Toucher une fenêtre. Numéro 3. Montrez-moi une table. Numéro 4. Toucher une porte. Numéro 5. Montrez-moi un cahier. Numéro 6. Montrez-moi un ordinateur. Numéro 7. Toucher une chaise. Numéro 8. Montrez-moi un stylo.')]


## Creating a embedding model

In [6]:
def make_embedder() :
     model_name = 'intfloat/multilingual-e5-base'
     return HuggingFaceEmbeddings(
        model_name=model_name,
    )

hf = make_embedder()

## Creating Vector Database

In [7]:
document = texts[0]
text = document.page_content  # Extracting text

print("Extracted Text:", text)

vectorstore2 = DocArrayInMemorySearch.from_documents(texts, hf)

Extracted Text: Numéro 1. Toucher un crayon. Numéro 2. Toucher une fenêtre. Numéro 3. Montrez-moi une table. Numéro 4. Toucher une porte. Numéro 5. Montrez-moi un cahier. Numéro 6. Montrez-moi un ordinateur. Numéro 7. Toucher une chaise. Numéro 8. Montrez-moi un stylo.


## Testing

In [9]:
source_language = "fr" #language
target_language = input("Enter the target language: ")

translation_chain = RunnablePassthrough() | prompt_translation | model | parser 
summary_chain = {"answer": translation_chain} | prompt_summary | model | parser 

try:
    result = translation_chain.invoke({
        "context": vectorstore2.as_retriever(),
        "text": text,
        "source_language": source_language,
        "target_language": target_language
    })
    
    print("Processed Result:", result)

    resultSum = summary_chain.invoke({
        "context": vectorstore2.as_retriever(),
        "text": result,
        "source_language": source_language,
        "target_language": target_language
    })
    print("\nSummary Result: ", resultSum)
    
except Exception as e:
    print(f"Error invoking chain: {e}")

Processed Result: To translate the given text from French to English, I'll proceed step by step.

Given text:
```
Numéro 1. Toucher un crayon.
Numéro 2. Toucher une fenêtre.
Numéro 3. Montrez-moi une table.
Numéro 4. Toucher une porte.
Numéro 5. Montrez-moi un cahier.
Numéro 6. Montrez-moi un ordinateur.
Numéro 7. Toucher une chaise.
Numéro 8. Montrez-moi un stylo.
```

Translation:
1. **Number 1. Touch a pencil.**
2. **Number 2. Touch a window.**
3. **Number 3. Show me a table.**
4. **Number 4. Touch a door.**
5. **Number 5. Show me a notebook.** ( Cahier can also mean "notebook" in French, or more generally it could refer to any kind of notebook but here given context suggests 'show me a notebook')
6. **Number 6. Show me a computer.**
7. **Number 7. Touch a chair.**
8. **Number 8. Show me a pen.** (Stylo in French can also mean "pen" as opposed to crayon or pencil, so the translation might slightly vary depending on whether you're referring specifically to a writing instrument that u